In [54]:
import time

from fmm import Fmm
from fmm.backend.numba import prepare_p2m_data, p2m_core, prepare_u_list_data

In [74]:
# ! rm test.hdf5 && fmm generate-test-data -c test && fmm compute-operators -c test

In [42]:
e = Fmm('test')

# P2M

In [93]:
p2m_org_time = []
p2m_calc_time = []

nruns = 5

for _ in range(nruns):
    s = time.time()

    scales, check_potentials = prepare_p2m_data(
        leaves=e.leaves,
        nleaves=e.nleaves,
        sources=e.sources,
        source_densities=e.source_densities,
        source_index_pointer=e.source_index_pointer,
        key_to_leaf_index=e.key_to_leaf_index,
        x0=e.x0,
        r0=e.r0,
        alpha_outer=e.alpha_outer,
        check_surface=e.check_surface,
        ncheck_points=e.ncheck_points,
        p2p_function=e.p2p_function,
        scale_function=e.scale_function
    )
    p2m_org_time.append(time.time()-s)

    s = time.time()

    p2m_core(
            leaves=e.leaves,
            nleaves=e.nleaves,
            key_to_index=e.key_to_index,
            nequivalent_points=e.nequivalent_points,
            ncheck_points=e.ncheck_points,
            uc2e_inv_a=e.uc2e_inv_a,
            uc2e_inv_b=e.uc2e_inv_b,
            scales=scales,
            multipole_expansions=e.multipole_expansions,
            check_potentials=check_potentials
        )

    p2m_calc_time.append(time.time()-s)
    
p2m_org_time = np.array(p2m_org_time)
p2m_calc_time = np.array(p2m_calc_time)

In [94]:
np.mean(p2m_org_time)

0.4411337375640869

In [95]:
np.std(p2m_org_time)

0.03153015325622455

In [96]:
np.mean(p2m_calc_time)

0.3298027992248535

In [97]:
np.std(p2m_calc_time)

0.012777502957859984

# P2P

In [98]:
nruns = 5
p2p_org_time = []
p2p_calc_time = []

for _ in range(nruns):
    
    s = time.time()
    
    local_sources, local_targets, local_source_densities, local_source_index_pointer, local_target_index_pointer = prepare_u_list_data(
            leaves=e.leaves,
            targets=e.targets,
            target_index_pointer=e.target_index_pointer,
            sources=e.sources,
            source_densities=e.source_densities,
            source_index_pointer=e.source_index_pointer,
            key_to_index=e.key_to_index,
            key_to_leaf_index=e.key_to_leaf_index,
            u_lists=e.u_lists,
            max_points=e.config['max_points'],
        )
    
    p2p_org_time.append(time.time()-s)
    
    s = time.time()

    target_potentials_vec = e.p2p_parallel_function(
        sources=local_sources,
        targets=local_targets,
        source_densities=local_source_densities,
        source_index_pointer=local_source_index_pointer,
        target_index_pointer=local_target_index_pointer
    )
    
    p2p_calc_time.append(time.time()-s)
    
    s = time.time()
    nleaves = len(local_target_index_pointer) - 1

    for i in range(nleaves):
        res = target_potentials_vec[local_target_index_pointer[i]:local_target_index_pointer[i+1]]
        leaf = e.leaves[i]
        leaf_idx = e.key_to_leaf_index[leaf]
        e.target_potentials[e.target_index_pointer[leaf_idx]:e.target_index_pointer[leaf_idx+1], :] += res
    
    p2p_org_time[-1]+= time.time()-s
    
    
p2p_calc_time = np.array(p2p_calc_time)
p2p_org_time = np.array(p2p_org_time)

In [103]:
e.nleaves

32768

In [99]:
np.mean(p2p_calc_time)

2.0889453411102297

In [100]:
np.std(p2p_calc_time)

0.0838038470299267

In [101]:
np.mean(p2p_org_time)

0.9084608078002929

In [102]:
np.std(p2p_org_time)

0.052282821069864406